<a href="https://colab.research.google.com/github/dmarchignoli/haystack-rag/blob/main/simple_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/dmarchignoli/haystack-rag.git

Cloning into 'haystack-rag'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 12 (delta 0), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 7.49 KiB | 7.49 MiB/s, done.


In [ ]:
%cd haystack-rag/

/content/haystack-rag


In [ ]:
!pip install haystack-ai sentence-transformers

In [ ]:
from haystack import Document

texts = """
There are over 7,000 languages spoken around the world today.
Elephants have been observed to behave in a way that indicates a high level of self-awareness, such as recognizing themselves in mirrors.
In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the phenomenon of bioluminescent waves.
""".strip().split('\n')

documents = [Document(content=x) for x in texts]

for d in documents:
  print(d)


Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.')
Document(id=6f20658aeac3c102495b198401c1c0c2bd71d77b915820304d4fbc324b2f3cdb, content: 'Elephants have been observed to behave in a way that indicates a high level of self-awareness, such ...')
Document(id=7f225626ad1019b273326fbaf11308edfca6d663308a4a3533ec7787367d59a2, content: 'In certain parts of the world, like the Maldives, Puerto Rico, and San Diego, you can witness the ph...')


In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder

model_name="HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1"
text_embedder = SentenceTransformersTextEmbedder(model=model_name)
document_embedder = SentenceTransformersDocumentEmbedder(model=model_name)

text_embedder.warm_up()
document_embedder.warm_up()

print(text_embedder.run(texts[0])['embedding'])
print(document_embedder.run(documents[:1]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/600k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.003011721186339855, -0.006784013472497463, -0.008190589025616646, -0.0049788192845880985, 0.004092305898666382, 7.894526788732037e-05, -0.008068577386438847, 0.0005798705387860537, -0.020810117945075035, 0.008193599060177803, -0.03229497745633125, 0.009576637297868729, -0.006836182903498411, -0.014892421662807465, -0.007112565916031599, 0.01798364892601967, -0.036985132843256, -0.021843060851097107, -0.03766950964927673, 0.0010961295338347554, -0.0641428753733635, -0.032925840467214584, -0.01206203643232584, 0.01574116013944149, 0.005689284764230251, 0.07852021604776382, 0.007477662526071072, 0.009653611108660698, 0.016210569068789482, -0.007366318721324205, -0.012833131477236748, -0.041939206421375275, 0.008757617324590683, -0.011009165085852146, 0.012921972200274467, 0.04111570864915848, 0.03296510502696037, 0.04146687313914299, 0.0200977586209774, -0.01398850791156292, -0.009523103944957256, 0.020560937002301216, 0.013438652269542217, 0.0009135245927609503, -0.02329723909497261, 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', embedding: vector of size 896)]}


In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryEmbeddingRetriever

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

documents_with_embeddings = document_embedder.run(documents)["documents"]
document_store.write_documents(documents_with_embeddings)
print(document_store.count_documents())
print(document_store.filter_documents({
    "field":"embedding", "operator":"==", "value":documents_with_embeddings[0].embedding}))

document_retriever = InMemoryEmbeddingRetriever(document_store=document_store)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

3
[Document(id=cfe93bc1c274908801e6670440bf2bbba54fad792770d57421f85ffa2a4fcc94, content: 'There are over 7,000 languages spoken around the world today.', embedding: vector of size 896)]


In [ ]:
from haystack import Pipeline

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder())
query_pipeline.add_component("retriever", )
query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

query = "How many languages are there?"

result = query_pipeline.run({"text_embedder": {"text": query}})

print(result['retriever']['documents'][0])